# Get entries from DB(sheets)

upload the secret_files for sheets access. **FD4GS.json**,**FD4GS_cache.json** in */content*

In [ ]:
! pip install gsheets gdown > /dev/null

In [ ]:
#google sheet package
from gsheets import Sheets
#download image from sheet
import gdown

#Authorizing the api
sheets = Sheets.from_files('FD4GS.json','FD4GS_cache.json')

In [ ]:
#Fetching information from helper database
vi3 = sheets.get('1dYaQqMVPEbJuBwHC-nPbu-2NPIcaqwJf8-MqiZswmU4') # Helper Database
vi3_form1_ws = vi3.sheets[0]

entries = vi3_form1_ws.values()[1:]
for i, e in enumerate(entries):
  gdown.download('https://drive.google.com/uc?id='+e[1].split('id=')[1], f'entrie_{i}', quiet= i!=0)

# Detect Vehicles and Get Features

In [ ]:
! git clone https://github.com/ProjectDetector/Train-VehicleDetection-model.git
! cd Train-VehicleDetection-model/model.weights_/ && sh to_merge.sh

!pip install intel-tensorflow > /dev/null

In [ ]:
mymodel = __import__('Train-VehicleDetection-model.model').model
model = mymodel.get_model()
mymodel.load_weights(model, 'Train-VehicleDetection-model/model.weights')

In [ ]:
def get_boxes(path, th=0.01, toplot=False):
  img = mymodel.mpimg.imread(path)
  pre_precessed = mymodel.preprocess(img)
  batch = mymodel.np.expand_dims(pre_precessed, axis=0)
  batch_output = model.predict(batch)
  boxes = mymodel.model_output_to_boxes(batch_output[0], threshold=th)
  if toplot: plot(img, boxes)
  return boxes

##### For testing

In [ ]:
rm -rf Train-VehicleDetection-model/

In [ ]:
import importlib
importlib.reload(mymodel)

In [ ]:
print(get_boxes('Train-VehicleDetection-model/testimg.jpg', 0.01, 1))
#print(get_boxes(f'entrie_{input()}', 0.01, 1))

In [ ]:
def draw_boxes(boxes,im, crop_dim):
    imgcv1 = im.copy()
    [xmin, xmax] = crop_dim[0]
    [ymin, ymax] = crop_dim[1]
    
    height, width, _ = imgcv1.shape
    for b in boxes:
        w = xmax - xmin
        h = ymax - ymin

        left  = int ((b.x - b.w/2.) * w) + xmin
        right = int ((b.x + b.w/2.) * w) + xmin
        top   = int ((b.y - b.h/2.) * h) + ymin
        bot   = int ((b.y + b.h/2.) * h) + ymin

        if left  < 0:
            left = 0
        if right > width - 1:
            right = width - 1
        if top < 0:
            top = 0
        if bot>height - 1: 
            bot = height - 1
        
        thick = 5 #int((height + width // 150))
        
        mymodel.cv2.rectangle(imgcv1, (left, top), (right, bot), (255,0,0), thick)

    return imgcv1

import matplotlib.pyplot as plt
%matplotlib inline

def plot(img, boxes):
  final = draw_boxes(boxes, img, ((500,1280),(300,650)))

  plt.rcParams['figure.figsize'] = (20, 10)
  plt.subplot(1,3,1)
  plt.imshow(get_croped_imgs(img, boxes)[0])#img)
  plt.axis('off')
  plt.title("Original Image")
  plt.subplot(1,3,2)
  plt.imshow(mymodel.normalize(mymodel.crop_and_resize(img)))
  plt.axis('off')
  plt.title("Pre Processed")
  plt.subplot(1,3,3)
  plt.imshow(final)
  plt.axis('off')
  plt.title("With Boxes")

# To Detect color

In [ ]:
def get_croped_imgs(img, boxes): # yet to code
  rtn = []
  for box in boxes:
    x = bx - box.w/2
    y = by - box.h/2
    x, y, h, w = int(x), int(y), int(box.h), int(box.w)
    print(x,y,h,w,box.x,box.y,box.h,box.w)
    rtn.append(img[y:y+h, x:x+w])
  return rtn

In [ ]:
# upload the colors.csv in /content

from PIL import Image
import cv2
import numpy as np
import pandas as pd
import scipy
import scipy.misc
import scipy.cluster


index=["color","color_name","hex","R","G","B"]
csv = pd.read_csv('colors.csv', names=index, header=None)

#function to calculate minimum distance from all colors and get the most matching color
def getColorName(R,G,B):
    minimum = 10000
    for i in range(len(csv)):
        d = abs(R- int(csv.loc[i,"R"])) + abs(G- int(csv.loc[i,"G"]))+ abs(B- int(csv.loc[i,"B"]))
        if(d<=minimum):
            minimum = d
            cname = csv.loc[i,"color_name"]
    return cname


NUM_CLUSTERS = 5

def getRGB(im):
    im = cv2.resize(im, (150,150)) #im.resize((150, 150))      # optional, to reduce time
    #ar = np.asarray(im)
    shape = ar.shape
    ar = ar.reshape(scipy.product(shape[:2]), shape[2]).astype(float)

    #print('finding clusters')
    codes, dist = scipy.cluster.vq.kmeans(ar, NUM_CLUSTERS)
    #print('cluster centres:\n', codes)

    vecs, dist = scipy.cluster.vq.vq(ar, codes)         # assign codes
    counts, bins = scipy.histogram(vecs, len(codes))    # count occurrences

    index_max = scipy.argmax(counts)                    # find most frequent
    peak = codes[index_max]
    r,g,b = (int(c) for c in peak)

    return r,g,b
